# Converter for the Financial Filings at the Sec

In [21]:
from zipfile import ZipFile
import pandas as pd

dirname = "2024q2"
with ZipFile('importFiles/'+dirname+'.zip') as myzip:
    with myzip.open('num.txt') as myfile1:
        dfNum = pd.read_table(myfile1,delimiter="\t", low_memory=False)
    with myzip.open('pre.txt') as myfile2:
        dfPre = pd.read_table(myfile2,delimiter="\t")
    with myzip.open('sub.txt') as myfile3:
        dfSub = pd.read_table(myfile3,delimiter="\t", low_memory=False)
    with myzip.open('tag.txt') as myfile4:
        dfTag = pd.read_table(myfile4,delimiter="\t")

dfSym = pd.read_table('importFiles/ticker.txt', delimiter="\t", header=None, names=['symbol','cik'])        

print(dfNum.size)
print(dfPre.size)
print(dfSub.size)
print(dfTag.size)
print(dfSym.size)

34261700
8384750
276300
879939
24168


In [23]:
from tabulate import tabulate

print(tabulate(dfNum.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+----------------------------------------------------------------------------------------------------------------------------------------------+----------------------+----------+--------+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+-------------------+------------+
|    | adsh                 | tag                                                                                                                                          | version              |    ddate |   qtrs | uom    | segments                                                                                                                                                                                             | coreg              |             value |   footnote |
|----+----------------------+---------------

In [24]:
print(tabulate(dfPre.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+----------+--------+--------+---------+---------+----------------------------------------------------------------------------------------------------------------+----------------------+----------------------------------------------------------------------------------------------------------------------+------------+
|    | adsh                 |   report |   line | stmt   |   inpth | rfile   | tag                                                                                                            | version              | plabel                                                                                                               |   negating |
|----+----------------------+----------+--------+--------+---------+---------+----------------------------------------------------------------------------------------------------------------+----------------------+--------------------------------------------------------------------------------------------

In [25]:
print(tabulate(dfSub.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+--------+--------------------------------------+-------+-------------+----------+-----------------+------------+-----------------------------------------+----------------------------------------+----------------+-------------+----------+-----------------+------------+-----------------------------------------+----------------------------------------+--------------+-----------+-----------+-------------------------------------------+---------------+-------+--------+-------+--------+-------------+------+------+----------+-----------------------+-----------+----------+---------------------------+---------+----------------------------------------------+
|    | adsh                 |    cik | name                                 |   sic | countryba   | stprba   | cityba          | zipba      | bas1                                    | bas2                                   | baph           | countryma   | stprma   | cityma          | zipma      | mas1             

In [26]:
print(tabulate(dfTag.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------------------------------------------------------------------------------------------------+--------------+----------+------------+------------+--------+--------+--------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    | ta

In [27]:
print(tabulate(dfSym.head(10), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------+---------+
|    | symbol   |     cik |
|----+----------+---------|
|  0 | aapl     |  320193 |
|  1 | msft     |  789019 |
|  2 | brk-b    | 1067983 |
|  3 | unh      |  731766 |
|  4 | jnj      |  200406 |
|  5 | v        | 1403161 |
|  6 | tsm      | 1046179 |
|  7 | xom      |   34088 |
|  8 | wmt      |  104169 |
|  9 | spy      |  884394 |
+----+----------+---------+
adsh         object
tag          object
version      object
ddate         int64
qtrs          int64
uom          object
segments     object
coreg        object
value       float64
footnote     object
dtype: object


In [28]:
print(dfNum.iloc[0])
print(dfNum.columns.values)

adsh                               0001104659-24-066053
tag         AdjustmentsForDecreaseIncreaseInInventories
version                                       ifrs/2023
ddate                                          20230331
qtrs                                                  3
uom                                                 USD
segments                                            NaN
coreg                                               NaN
value                                       -19371086.0
footnote                                            NaN
Name: 0, dtype: object
['adsh' 'tag' 'version' 'ddate' 'qtrs' 'uom' 'segments' 'coreg' 'value'
 'footnote']


In [29]:
from datetime import date, datetime, timedelta
from marshmallow import Schema, fields

class FinancialElementImportDto:
    label = ""          #Tag.doc
    concept = ""        #Num.tag
    info = ""           #Pre.plabel
    unit = ""           #Num.uom
    value = 0.0         #Num.value

class FinancialsDataDto:
    bs = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    cf = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    ic = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    

class SymbolFinancialsDto:
    startDate = date.today()   #Sub.period 
    endDate = date.today()     #Sub.period + Sub.fp
    year = 0                   #Sub.fy
    quarter = ""               #Sub.fp
    symbol = ""                #Sub.cik -> Sym.cik -> Sym.symbol
    name = ""                  #Sub.name
    country = ""               #Sub.countryma
    city = ""                  #Sub.cityma
    data = FinancialsDataDto()
    
class FinancialElementImportSchema(Schema):
    label = fields.String()
    concept = fields.String()
    info = fields.String()
    unit = fields.String()
    value = fields.Int()
    
class FinancialsDataSchema(Schema):
    bs = fields.List(fields.Nested(FinancialElementImportSchema()))
    cf = fields.List(fields.Nested(FinancialElementImportSchema()))
    ic = fields.List(fields.Nested(FinancialElementImportSchema()))
    
class SymbolFinancialsSchema(Schema):
    startDate = fields.DateTime()
    endDate = fields.DateTime()
    year = fields.Int()
    quarter = fields.String()
    symbol = fields.String()
    name = fields.String()
    country = fields.String()
    city = fields.String()
    data = fields.Nested(FinancialsDataSchema)

In [30]:
import os

#isExist = os.path.exists("exportFiles/"+dirname)
isExist = 1 == 2
if isExist:
    for filename in os.listdir("exportFiles/"+dirname):
        with open(os.path.join("exportFiles/"+dirname, filename), 'r') as file:
            jsonStr = file.read()
            file.close()
        jsonStr = jsonStr.replace("'", '"').replace(", ",",").replace('\\r', '').replace('\\n', ' ')
        jsonStr = ''.join(jsonStr.splitlines())
        with open(os.path.join("exportFiles/"+dirname, filename), 'w') as file:    
            file.write(jsonStr)
            file.close()
    

# Create a single SymbolFinancialsDto

In [31]:
#json_string = json.dumps([ob.__dict__ for ob in list_name])
#rslt_df = dataframe[dataframe['Percentage'] > 80]
import numpy as np
from pprint import pprint
import os
from dateutil.relativedelta import relativedelta
import json

def npInt_to_str(var):
    return str(list(np.reshape(np.asarray(var), (1, np.size(var)))[0]))[1:-1]

def formatDateNpNum(var):
    dateStr = npInt_to_str(var)
    return dateStr[0:4]+"-"+dateStr[4:6]+"-"+dateStr[6:8]

fileStartTime = datetime.now()
for subId in range(len(dfSub)):
    startTime = datetime.now()
    submitted = dfSub.iloc[subId]
    sfDto = SymbolFinancialsDto()
    sfDto.data = FinancialsDataDto()
    sfDto.data.bs = []
    sfDto.data.cf = []
    sfDto.data.ic = []
    try:
        periodStartDate = date.fromisoformat(formatDateNpNum(submitted["period"]))
    except ValueError:
        print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Period: "+str(submitted["period"]))
        continue       
    sfDto.startDate = periodStartDate
    sfDto.endDate = date.today() #default value
    if submitted["fy"] == np.nan or np.isnan(submitted["fy"]):
        sfDto.year = 0
    else: 
        sfDto.year = submitted["fy"].astype(int)        
    sfDto.quarter = str(submitted["fp"]).strip().upper()
    
    if sfDto.quarter == "FY" or sfDto.quarter == "CY":
        sfDto.endDate = periodStartDate + relativedelta(months=+12, days=-1)
    elif sfDto.quarter == "H1" or sfDto.quarter == "H2":
        sfDto.endDate = periodStartDate + relativedelta(months=+6, days=-1)
    elif sfDto.quarter == "T1" or sfDto.quarter == "T2" or sfDto.quarter == "T3":
        sfDto.endDate = periodStartDate + relativedelta(months=+4, days=-1)
    elif sfDto.quarter == "Q1" or sfDto.quarter == "Q2" or sfDto.quarter == "Q3" or sfDto.quarter == "Q4":
        sfDto.endDate = periodStartDate + relativedelta(months=+3, days=-1)
    else:
        continue
        
    #symStr = dfSym[dfSym["cik"]==submitted.cik].symbol.str.upper().astype("string")
    #print(symStr)
    val = dfSym[dfSym["cik"]==submitted["cik"]]
    if len(val) > 0:
        sfDto.symbol = val["symbol"].to_string(index = False).split("\n")[0].split("\\n")[0].strip().split(" ")[0].strip().upper()
        if len(sfDto.symbol) > 19 or len(sfDto.symbol) < 1:
            print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Symbol "+sfDto.symbol)
    else:
        print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Symbol "+val["symbol"].to_string(index = False))
        continue
        
    sfDto.name = submitted["name"]
    sfDto.country = submitted["countryma"]
    sfDto.city = submitted["cityma"]

    sdDto = FinancialsDataDto()
    #print(filteredDfPre)
    dfNum['value'] = pd.to_numeric(dfNum['value'], errors='coerce')
    dfNum = dfNum.dropna(subset=['value'])
    dfNum['value'] = dfNum['value'].astype(int)
    #filteredDfNum = dfNum[dfNum['adsh'] == submitted['adsh']]       
    filteredDfNum = dfNum.loc[dfNum['adsh'].values == submitted['adsh']]
    filteredDfNum.reset_index()

    for myId in range(len(filteredDfNum)): 
        #print(myId)
        #print(filteredDfNum.iloc[myId])
        myNum = filteredDfNum.iloc[myId]
        myDto = FinancialElementImportDto()
        myTag = dfTag[dfTag["tag"] == myNum['tag']]
        myDto.label = myTag["doc"].to_string(index = False)
        myDto.concept = myNum["tag"]
        #myPre = dfPre[(dfPre['adsh'] == submitted["adsh"]) & (dfPre['tag'] == myNum['tag'])]
        myPre = dfPre.loc[np.where((dfPre['adsh'].values == submitted["adsh"]) & (dfPre['tag'].values == myNum['tag']))]
        myDto.info = myPre["plabel"].to_string(index = False).replace('"',"'")
        myDto.unit = myNum["uom"]
        #print(myNum["value"])        
        myDto.value = myNum["value"]
        #print(myDto.__dict__)
        #FinancialElementImportSchema().dump(myDto)
        if myPre['stmt'].to_string(index = False).strip() == 'BS':
            sfDto.data.bs.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'CF':
            sfDto.data.cf.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'IC':
            sfDto.data.ic.append(myDto)

    #print(len(filteredDfNum))
    #print(submitted)
    #print(sfDto.__dict__)    
    #json.dumps(sfDto.__dict__)    
    marshmellowStart = datetime.now()
    result = SymbolFinancialsSchema().dump(sfDto)
    result = str(json.dumps(result))
    #result = result.replace('"','öäü').replace("'", '"').replace("öäü","'").replace(", ",",")
    result = result.replace('\\r', '').replace('\\n', ' ')
    
    print("FilterDfNum size: "+str(len(filteredDfNum))+" Dtos size: "+str((len(sfDto.data.bs)+len(sfDto.data.cf)+len(sfDto.data.ic)))+" Bs: "+str(len(sfDto.data.bs))+" Cf: "+str(len(sfDto.data.cf))+" Ic: "+str(len(sfDto.data.ic))+" Json size: "+str(int(len(str(result)) / 1024))+"kb.")
    
    #print("Json size "+str(int(len(str(result)) / 1024))+"kb. Time: "+str((datetime.now() - marshmellowStart) / timedelta(milliseconds=1))+"ms.")
    
    #print(result)
    #pprint(result)    
    isExist = os.path.exists("exportFiles/"+dirname)
    if not isExist:
        os.makedirs("exportFiles/"+dirname)
    json_file = open("exportFiles/"+dirname+"/"+submitted["adsh"]+".json","w")
    json_file.write(result)
    json_file.close()
    endTime = datetime.now()
    print("file "+str(subId+1)+" of "+str(len(dfSub)+1)+" exportFiles/"+dirname+"/"+submitted["adsh"]+".json stored in "+str((endTime - startTime) / timedelta(milliseconds=1))+"ms.")
    
fileEndTime = datetime.now()
print("Time "+str((fileEndTime - fileStartTime) / timedelta(seconds=1))+"sec for "+str(len(dfSub)+1)+" files.")

FilterDfNum size: 291 Dtos size: 133 Bs: 87 Cf: 46 Ic: 0 Json size: 37kb.
file 1 of 7676 exportFiles/2024q2/0000002178-24-000054.json stored in 10820.326ms.
FilterDfNum size: 247 Dtos size: 125 Bs: 70 Cf: 55 Ic: 0 Json size: 35kb.
file 2 of 7676 exportFiles/2024q2/0000002488-24-000056.json stored in 8657.662ms.
FilterDfNum size: 786 Dtos size: 223 Bs: 136 Cf: 87 Ic: 0 Json size: 63kb.
file 3 of 7676 exportFiles/2024q2/0000002969-24-000026.json stored in 31661.736ms.
FilterDfNum size: 204 Dtos size: 115 Bs: 73 Cf: 42 Ic: 0 Json size: 31kb.
file 4 of 7676 exportFiles/2024q2/0000003499-24-000015.json stored in 8463.898ms.
FilterDfNum size: 427 Dtos size: 176 Bs: 100 Cf: 76 Ic: 0 Json size: 49kb.
file 5 of 7676 exportFiles/2024q2/0000003545-24-000066.json stored in 16761.331ms.
FilterDfNum size: 426 Dtos size: 210 Bs: 148 Cf: 62 Ic: 0 Json size: 78kb.
file 6 of 7676 exportFiles/2024q2/0000003570-24-000055.json stored in 17188.146ms.
FilterDfNum size: 378 Dtos size: 140 Bs: 74 Cf: 66 Ic: 0 

FilterDfNum size: 567 Dtos size: 307 Bs: 241 Cf: 66 Ic: 0 Json size: 79kb.
file 53 of 7676 exportFiles/2024q2/0000021175-24-000039.json stored in 18675.628ms.
FilterDfNum size: 301 Dtos size: 167 Bs: 107 Cf: 60 Ic: 0 Json size: 48kb.
file 54 of 7676 exportFiles/2024q2/0000021344-24-000017.json stored in 9497.283ms.
FilterDfNum size: 285 Dtos size: 130 Bs: 70 Cf: 60 Ic: 0 Json size: 36kb.
file 55 of 7676 exportFiles/2024q2/0000021665-24-000015.json stored in 9642.458ms.
FilterDfNum size: 547 Dtos size: 310 Bs: 242 Cf: 68 Ic: 0 Json size: 86kb.
file 56 of 7676 exportFiles/2024q2/0000022356-24-000035.json stored in 22710.909ms.
FilterDfNum size: 571 Dtos size: 161 Bs: 101 Cf: 60 Ic: 0 Json size: 47kb.
file 57 of 7676 exportFiles/2024q2/0000022444-24-000089.json stored in 21424.117ms.
FilterDfNum size: 389 Dtos size: 200 Bs: 120 Cf: 80 Ic: 0 Json size: 54kb.
file 58 of 7676 exportFiles/2024q2/0000022701-24-000002.json stored in 12810.222ms.
FilterDfNum size: 320 Dtos size: 165 Bs: 108 Cf: 

FilterDfNum size: 373 Dtos size: 128 Bs: 58 Cf: 70 Ic: 0 Json size: 39kb.
file 107 of 7676 exportFiles/2024q2/0000041719-24-000018.json stored in 11729.379ms.
FilterDfNum size: 228 Dtos size: 116 Bs: 70 Cf: 46 Ic: 0 Json size: 33kb.
file 108 of 7676 exportFiles/2024q2/0000042888-24-000014.json stored in 7348.756ms.
FilterDfNum size: 703 Dtos size: 211 Bs: 128 Cf: 83 Ic: 0 Json size: 65kb.
file 109 of 7676 exportFiles/2024q2/0000043920-24-000030.json stored in 23186.935ms.
FilterDfNum size: 245 Dtos size: 138 Bs: 96 Cf: 42 Ic: 0 Json size: 40kb.
file 110 of 7676 exportFiles/2024q2/0000045012-24-000015.json stored in 7844.457ms.
FilterDfNum size: 537 Dtos size: 195 Bs: 113 Cf: 82 Ic: 0 Json size: 73kb.
file 111 of 7676 exportFiles/2024q2/0000045876-24-000026.json stored in 18009.068ms.
FilterDfNum size: 319 Dtos size: 156 Bs: 82 Cf: 74 Ic: 0 Json size: 50kb.
file 112 of 7676 exportFiles/2024q2/0000045919-24-000014.json stored in 15955.263ms.
FilterDfNum size: 271 Dtos size: 126 Bs: 78 Cf

FilterDfNum size: 448 Dtos size: 224 Bs: 160 Cf: 64 Ic: 0 Json size: 57kb.
file 159 of 7676 exportFiles/2024q2/0000064803-24-000016.json stored in 14828.528ms.
FilterDfNum size: 321 Dtos size: 236 Bs: 176 Cf: 60 Ic: 0 Json size: 60kb.
file 160 of 7676 exportFiles/2024q2/0000064996-24-000019.json stored in 11864.458ms.
FilterDfNum size: 2111 Dtos size: 1221 Bs: 762 Cf: 459 Ic: 0 Json size: 333kb.
file 161 of 7676 exportFiles/2024q2/0000065984-24-000049.json stored in 64891.07ms.
FilterDfNum size: 635 Dtos size: 154 Bs: 99 Cf: 55 Ic: 0 Json size: 43kb.
file 162 of 7676 exportFiles/2024q2/0000066382-24-000014.json stored in 20014.932ms.
FilterDfNum size: 281 Dtos size: 149 Bs: 89 Cf: 60 Ic: 0 Json size: 42kb.
file 163 of 7676 exportFiles/2024q2/0000066570-24-000022.json stored in 12136.331ms.
FilterDfNum size: 285 Dtos size: 188 Bs: 134 Cf: 54 Ic: 0 Json size: 54kb.
file 164 of 7676 exportFiles/2024q2/0000066740-24-000053.json stored in 9739.457ms.
FilterDfNum size: 326 Dtos size: 186 Bs:

FilterDfNum size: 340 Dtos size: 146 Bs: 94 Cf: 52 Ic: 0 Json size: 40kb.
file 213 of 7676 exportFiles/2024q2/0000084748-24-000028.json stored in 11489.201ms.
FilterDfNum size: 264 Dtos size: 150 Bs: 88 Cf: 62 Ic: 0 Json size: 43kb.
file 214 of 7676 exportFiles/2024q2/0000084839-24-000070.json stored in 8501.508ms.
FilterDfNum size: 373 Dtos size: 223 Bs: 174 Cf: 49 Ic: 0 Json size: 59kb.
file 215 of 7676 exportFiles/2024q2/0000085535-24-000010.json stored in 12489.243ms.
FilterDfNum size: 350 Dtos size: 129 Bs: 69 Cf: 60 Ic: 0 Json size: 47kb.
file 216 of 7676 exportFiles/2024q2/0000085961-24-000078.json stored in 11993.054ms.
FilterDfNum size: 497 Dtos size: 255 Bs: 189 Cf: 66 Ic: 0 Json size: 70kb.
file 217 of 7676 exportFiles/2024q2/0000086312-24-000032.json stored in 17515.215ms.
FilterDfNum size: 358 Dtos size: 185 Bs: 133 Cf: 52 Ic: 0 Json size: 53kb.
file 218 of 7676 exportFiles/2024q2/0000088121-24-000060.json stored in 12491.313ms.
FilterDfNum size: 345 Dtos size: 149 Bs: 84 

...

FilterDfNum size: 267 Dtos size: 135 Bs: 75 Cf: 60 Ic: 0 Json size: 38kb.
file 7345 of 7676 exportFiles/2024q2/0001828318-24-000066.json stored in 5903.105ms.
FilterDfNum size: 643 Dtos size: 322 Bs: 247 Cf: 75 Ic: 0 Json size: 85kb.
file 7346 of 7676 exportFiles/2024q2/0001828376-24-000014.json stored in 13703.609ms.
FilterDfNum size: 244 Dtos size: 139 Bs: 89 Cf: 50 Ic: 0 Json size: 41kb.
file 7347 of 7676 exportFiles/2024q2/0001828536-24-000099.json stored in 5362.254ms.
FilterDfNum size: 402 Dtos size: 149 Bs: 62 Cf: 87 Ic: 0 Json size: 40kb.
file 7348 of 7676 exportFiles/2024q2/0001828723-24-000065.json stored in 8136.468ms.
FilterDfNum size: 293 Dtos size: 139 Bs: 82 Cf: 57 Ic: 0 Json size: 40kb.
file 7349 of 7676 exportFiles/2024q2/0001828791-24-000024.json stored in 5963.826ms.
FilterDfNum size: 377 Dtos size: 201 Bs: 112 Cf: 89 Ic: 0 Json size: 76kb.
file 7350 of 7676 exportFiles/2024q2/0001828852-24-000076.json stored in 7639.525ms.
FilterDfNum size: 461 Dtos size: 103 Bs: 53

FilterDfNum size: 126 Dtos size: 76 Bs: 60 Cf: 16 Ic: 0 Json size: 22kb.
file 7408 of 7676 exportFiles/2024q2/0001829126-24-003569.json stored in 2739.585ms.
FilterDfNum size: 178 Dtos size: 94 Bs: 56 Cf: 38 Ic: 0 Json size: 24kb.
file 7409 of 7676 exportFiles/2024q2/0001829126-24-003664.json stored in 3779.823ms.
FilterDfNum size: 404 Dtos size: 179 Bs: 95 Cf: 84 Ic: 0 Json size: 50kb.
file 7410 of 7676 exportFiles/2024q2/0001829126-24-003713.json stored in 8246.37ms.
FilterDfNum size: 340 Dtos size: 20 Bs: 2 Cf: 18 Ic: 0 Json size: 5kb.
file 7413 of 7676 exportFiles/2024q2/0001829126-24-003896.json stored in 6997.958ms.
FilterDfNum size: 130 Dtos size: 76 Bs: 49 Cf: 27 Ic: 0 Json size: 33kb.
file 7414 of 7676 exportFiles/2024q2/0001829126-24-003898.json stored in 2834.784ms.
FilterDfNum size: 321 Dtos size: 141 Bs: 76 Cf: 65 Ic: 0 Json size: 42kb.
file 7415 of 7676 exportFiles/2024q2/0001829126-24-003901.json stored in 6643.527ms.
FilterDfNum size: 261 Dtos size: 140 Bs: 83 Cf: 57 Ic

FilterDfNum size: 114 Dtos size: 64 Bs: 44 Cf: 20 Ic: 0 Json size: 19kb.
file 7469 of 7676 exportFiles/2024q2/0001842952-24-000035.json stored in 2473.427ms.
FilterDfNum size: 427 Dtos size: 218 Bs: 152 Cf: 66 Ic: 0 Json size: 88kb.
file 7470 of 7676 exportFiles/2024q2/0001844452-24-000070.json stored in 8492.566ms.
FilterDfNum size: 287 Dtos size: 145 Bs: 81 Cf: 64 Ic: 0 Json size: 48kb.
file 7471 of 7676 exportFiles/2024q2/0001844505-24-000013.json stored in 5816.495ms.
FilterDfNum size: 243 Dtos size: 126 Bs: 64 Cf: 62 Ic: 0 Json size: 50kb.
file 7472 of 7676 exportFiles/2024q2/0001845022-24-000072.json stored in 4967.084ms.
FilterDfNum size: 339 Dtos size: 170 Bs: 116 Cf: 54 Ic: 0 Json size: 73kb.
file 7473 of 7676 exportFiles/2024q2/0001845437-24-000018.json stored in 6877.619ms.
FilterDfNum size: 326 Dtos size: 176 Bs: 58 Cf: 118 Ic: 0 Json size: 52kb.
file 7474 of 7676 exportFiles/2024q2/0001845459-24-000007.json stored in 6703.388ms.
FilterDfNum size: 333 Dtos size: 161 Bs: 52 

FilterDfNum size: 283 Dtos size: 118 Bs: 72 Cf: 46 Ic: 0 Json size: 32kb.
file 7527 of 7676 exportFiles/2024q2/0001872789-24-000019.json stored in 5862.863ms.
FilterDfNum size: 241 Dtos size: 105 Bs: 65 Cf: 40 Ic: 0 Json size: 29kb.
file 7528 of 7676 exportFiles/2024q2/0001873923-24-000061.json stored in 5022.717ms.
FilterDfNum size: 210 Dtos size: 118 Bs: 72 Cf: 46 Ic: 0 Json size: 33kb.
file 7529 of 7676 exportFiles/2024q2/0001874178-24-000042.json stored in 4412.547ms.
FilterDfNum size: 583 Dtos size: 323 Bs: 240 Cf: 83 Ic: 0 Json size: 97kb.
file 7530 of 7676 exportFiles/2024q2/0001874474-24-000013.json stored in 11571.269ms.
FilterDfNum size: 306 Dtos size: 144 Bs: 66 Cf: 78 Ic: 0 Json size: 44kb.
file 7531 of 7676 exportFiles/2024q2/0001874944-24-000047.json stored in 6180.137ms.
FilterDfNum size: 586 Dtos size: 372 Bs: 291 Cf: 81 Ic: 0 Json size: 101kb.
file 7532 of 7676 exportFiles/2024q2/0001875246-24-000029.json stored in 11741.072ms.
FilterDfNum size: 465 Dtos size: 300 Bs: 

FilterDfNum size: 332 Dtos size: 124 Bs: 80 Cf: 44 Ic: 0 Json size: 36kb.
file 7590 of 7676 exportFiles/2024q2/0001929980-24-000138.json stored in 6844.694ms.
FilterDfNum size: 229 Dtos size: 127 Bs: 80 Cf: 47 Ic: 0 Json size: 38kb.
file 7591 of 7676 exportFiles/2024q2/0001929980-24-000160.json stored in 4768.444ms.
File exportFiles/2024q2/0001929980-24-000167.json has Symbol Series([], )
File exportFiles/2024q2/0001929980-24-000182.json has Symbol Series([], )
FilterDfNum size: 164 Dtos size: 92 Bs: 68 Cf: 24 Ic: 0 Json size: 28kb.
file 7594 of 7676 exportFiles/2024q2/0001929980-24-000194.json stored in 3488.114ms.
File exportFiles/2024q2/0001929980-24-000206.json has Symbol Series([], )
FilterDfNum size: 178 Dtos size: 105 Bs: 79 Cf: 26 Ic: 0 Json size: 29kb.
file 7596 of 7676 exportFiles/2024q2/0001929980-24-000217.json stored in 3772.927ms.
File exportFiles/2024q2/0001930087-24-000027.json has Symbol Series([], )
File exportFiles/2024q2/0001932393-24-000034.json has Symbol Series([